In [1]:
import torch
from torchvision import models, transforms, datasets
from PIL import Image 

In [2]:
#******************************
# Define image transformations
#******************************
img_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'valid' : transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'test'  : transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
}


In [6]:
#****************************************************
# Define dataset directory paths and other variables
#****************************************************
#Paths to be used on Google colab
# train_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/train'
# valid_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/valid'
# test_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/test'

#Paths to be used on local machine
train_datadir = '../data/TransferLearning/train'
valid_datadir = '../data/TransferLearning/valid'
test_datadir = '../data/TransferLearning/test'

data = {
    'train' : datasets.ImageFolder(root=train_datadir,transform=img_transforms['train']),
    'valid' : datasets.ImageFolder(root=valid_datadir,transform=img_transforms['valid']),
    'test'  : datasets.ImageFolder(root=test_datadir,transform=img_transforms['test'])
}

train_datalen = len(data['train'])
valid_datalen = len(data['valid'])
test_datalen = len(data['test'])

#Load the dataset in batches using the DataLoader
batch_len = 32
train_dataset = torch.utils.data.DataLoader(data['train'], batch_size= batch_len, shuffle= True)
valid_dataset = torch.utils.data.DataLoader(data['valid'], batch_size= batch_len, shuffle= True)
test_dataset  = torch.utils.data.DataLoader(data['test'], batch_size= batch_len, shuffle= True)



In [7]:
#Load the pre-trained model
model_resnet50 = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\R. Jagannathan/.cache\torch\checkpoints\resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 11.1MB/s]


In [20]:
#For transfer Learning we freeze the Feature extraction layer
#Set requires_grad = False to freeze the parameters and 
#the gradients are not computed in the backward() 
for param in model_resnet50.parameters():
    param.requires_grad = False

#Customize the final fully connected layer to fit our dataset
# fully_connected_input = model_resnet50.fc.in_features
model_resnet50.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(256, 10)
)


#Define the loss function and the optimizer
loss_func = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model_resnet50.parameters())
